In [ ]:
# !pip list

In [1]:
!pip install -qqq transformers
!pip install -qqq datasets

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 290 kB 4.1 MB/s 
     |████████████████████████████████| 243 kB 38.1 MB/s 
     |████████████████████████████████| 125 kB 49.4 MB/s 
     |████████████████████████████████| 1.1 MB 42.8 MB/s 
     |████████████████████████████████| 271 kB 49.3 MB/s 
     |████████████████████████████████| 160 kB 45.5 MB/s 
     |████████████████████████████████| 192 kB 42.5 MB/s 


In [2]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from transformers import BertModel, BertConfig, BertTokenizer, BertForSequenceClassification


In [3]:
# load  dataset and split the data into train/validation/test datasets
from datasets import load_dataset, DatasetDict

raw_datasets = load_dataset("financial_phrasebank", "sentences_50agree")
# 90% train and 10% test + validation
train_test_ds = raw_datasets["train"].train_test_split(test_size=0.1)

# Split the 10% test + valid in half test, half valid
test_valid = train_test_ds['test'].train_test_split(test_size=0.5)

# Gather everything into a single DatasetDict
dataset = DatasetDict({
    'train': train_test_ds['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
dataset

Downloading:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4361
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 243
    })
    valid: Dataset({
        features: ['sentence', 'label'],
        num_rows: 242
    })
})

In [4]:
dataset["train"][0]

{'label': 2, 'sentence': 'Cargo volume increased by approximately 5 % .'}

The labels are already in integers. To know the corresponding label to the integer, use features to inspect the dataset.

In [5]:
dataset["train"].features

{'label': ClassLabel(num_classes=3, names=['negative', 'neutral', 'positive'], names_file=None, id=None),
 'sentence': Value(dtype='string', id=None)}

### Preprocess the dataset
Convert the text to numbers the model can make sense of. Thsi can be done using Tokenizer

In [9]:
from transformers import BertTokenizer, DataCollatorWithPadding

checkpoint= "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(checkpoint)
inputs = tokenizer(dataset["train"]["sentence"])

def tokenize_function(example):
    return tokenizer(example["sentence"], padding = True, truncation = True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /root/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-6893cd3f0794b36a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-2adb7bb9d07b3050.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/financial_phrasebank/sentences_50agree/1.0.0/a6d468761d4e0c8ae215c77367e1092bead39deb08fbf4bffd7c0a6991febbf0/cache-af976b7042f3af6c.arrow


## Training

In [10]:
tokenized_datasets = tokenized_datasets.remove_columns(
    ["sentence"]
)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [11]:
# Define dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["valid"], batch_size=8, collate_fn=data_collator
)

In [12]:
#inspecting dataloaders
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 150]),
 'input_ids': torch.Size([8, 150]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 150])}

In [13]:
# Define the model 
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [14]:
# Pass batch to the model
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.2860, grad_fn=<NllLossBackward>) torch.Size([8, 3])


In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [16]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
print(num_training_steps)

1638


## The training loop

In [17]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [18]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch )
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1638 [00:00<?, ?it/s]

In [55]:
from sklearn.metrics import classification_report
pred_list = []

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        pred_list.append(predictions.cpu().numpy())
        
pred_list = np.concatenate(pred_list, axis =0)
pred_list


array([0, 2, 1, 1, 0, 2, 2, 0, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 0, 2, 1, 0, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 2, 1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2,
       2, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2,
       2, 1, 0, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2,
       2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 1, 2, 1, 2, 1, 0,
       1, 1, 1, 2, 0, 1, 1, 2, 2, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1, 2, 1, 1,
       0, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 1, 1, 2, 2, 1, 0, 1, 2, 1, 1, 1])

In [47]:
true_list = dataset["valid"]["label"]

In [56]:
print(classification_report(true_list,pred_list))

              precision    recall  f1-score   support

           0       0.86      0.83      0.84        23
           1       0.88      0.88      0.88       147
           2       0.78      0.81      0.79        72

    accuracy                           0.85       242
   macro avg       0.84      0.84      0.84       242
weighted avg       0.85      0.85      0.85       242



In [57]:
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)


In [59]:
pred_test = []
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        pred_test.append(predictions.cpu().numpy())
        
pred_test = np.concatenate(pred_test, axis =0)
pred_test

array([1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1,
       1, 0, 0, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2,
       1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 2,
       2, 2, 2, 1, 2, 1, 1, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 0, 2, 2, 2, 1, 1, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 2, 1, 2, 1, 1, 2, 1, 2, 1,
       2, 0, 2, 1, 1, 1, 0, 0, 1, 0, 2, 1, 0, 2, 1, 2, 1, 1, 1, 2, 2, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 1, 0, 1, 0, 1, 1, 1, 1, 2,
       0])

In [60]:
print(classification_report(dataset["test"]["label"],pred_test))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81        24
           1       0.95      0.88      0.91       160
           2       0.74      0.83      0.78        59

    accuracy                           0.87       243
   macro avg       0.81      0.86      0.83       243
weighted avg       0.88      0.87      0.87       243

